In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
headers = {"Accept-Language": "en-US,en;q=0.5"}
url = 'https://www.popvortex.com/music/charts/top-100-songs.php'

response = requests.get(url,headers=headers)
response.status_code 
soup = BeautifulSoup(response.content, "html.parser")

In [71]:
genre_list = soup.select("div.chart-wrapper div.chart-content ul li")

In [100]:
#initialize empty lists
title = []
singers = []
genre = []

# define the number of iterations of our for loop
# by checking how many elements are in the retrieved result set
# (this is equivalent but more robust than just explicitly defining 250 iterations)
num_iter = len(soup.select("div.chart-wrapper div.chart-content"))

title_list = soup.select("div.chart-wrapper div.chart-content p.title-artist cite.title")
singer_list = soup.select("div.chart-wrapper div.chart-content p.title-artist em.artist")

genre_list = soup.select("div.chart-wrapper div.chart-content ul li")
# removing extra elements that are not the genre
for i in reversed(range(len(genre_list))):
    if 'Genre' in str(genre_list[i]):
        pass
    else:
        del genre_list[i]
# removing extra text that is not the genre
for i in range(len(genre_list)):
    genre_list[i] = genre_list[i].get_text().replace('Genre: ','')
      

# iterate through the result set and retrive all the data
for i in range(num_iter):
    title.append(title_list[i].get_text())
    singers.append(singer_list[i].get_text().split('&'))
    genre.append(genre_list[i].split('/'))


# each list becomes a column
songs = pd.DataFrame({"title":title,
                       "singers":singers,
                       "genre":genre
                      })

songs

,title,singers,genre
0,Unholy,"[Sam Smith , Kim Petras]",[Pop]
1,Everywhere,[Fleetwood Mac],[Rock]
2,Eagle (feat. KB),[Transformation Worship],"[Hip-Hop , Rap]"
3,I'm Good (Blue),"[David Guetta , Bebe Rexha]",[Dance]
4,A Thousand Years,[Christina Perri],[Soundtrack]
...,...,...,...
95,Save Me,[Jelly Roll],[Rap]
96,Annie's Song,[John Denver],[Folk]
97,Lady Marmalade,"[Christina Aguilera, Lil' Kim, Mýa , P!nk]",[Soundtrack]
98,'Til You Can't,[Cody Johnson],[Country]


In [176]:
def recommend_song(record):
    # first I check if the song is in the top 100
    if record in songs['title'].values:
        # I create an empty dataframe for recommendations
        recommendations = pd.DataFrame()
        # I get a list of list of artists
        record_artists = songs[songs['title']==record]['singers']
        # I get a list of list of genres
        record_genres = songs[songs['title']==record]['genre']
        
        # For every artist in my list of artists
        for artist in record_artists[0]:
            # I add to my recommendations everything with this artist
            recommendations = pd.concat([recommendations,songs[songs['singers'].apply(lambda x: any([artist == singer for singer in x]))]],axis=0)
        
        # For every genre in my list of genres
        for genre in record_genres[0]:
            # I add to my recommendations everything containing this genre
            recommendations = pd.concat([recommendations,songs[songs['genre'].apply(lambda x: any([genre == genero for genero in x]))]],axis=0)               
                
        # dropping duplicates because we my code can easily create them
        recommendations= recommendations.drop_duplicates(subset=['title'])
        
        # Dropping the record we were given because we don't want to recommend the same
        recommendations = recommendations[recommendations['title']!=record]
        return recommendations.iloc[np.random.randint(0,high=len(recommendations))]
        
    else:
        return "We don't have a recommendation for you."

In [179]:
recommend_song('Unholy')

title      STAR WALKIN' (League of Legends Worlds Anthem)
singers                                       [Lil Nas X]
genre                                               [Pop]
Name: 45, dtype: object

# not finished yet  - Lab | Web Scraping Multiple Pages